## Homework 4

In [134]:
import pandas as pd
import numpy as np

1. (9 points) Read the `Loan.csv` data set into Python. This data set is for binary classification, predicting a loan default, or no default. The labels the data set are contained in the column `default`.

* **(a)** Write functions to carry out batch gradient descent, as described in class, for a logistic model on this data. You should include a function to compute the gradient of the loss, given the parameters. Another function, which uses the gradient function, should carry out the updates. A cutoff on the maximum number of iterations can be used, but there should also be a check for the stopping criterion discussed in class.
* **(b)** Use the functions from (a) to train a logistic model. Use a learning rate of 0.7 and initial parameters that are all 0.
* **(c)** If the output of the logistic model is 0.5 or larger, predict label 1, and label 0 otherwise. Determine the accuracy of your model on the data.

In [238]:
loan = pd.read_csv("Loan.csv")

In [239]:
# (a) functions for batch gradient decent 

def sig(z):
    return 1/(1+np.exp(-z)).reshape(-1,1)

def grad(x,y,w):
    n = x.shape[0]
    X = np.hstack((x,np.ones(n).reshape(-1,1)))
    f = sig(X @ w)
    return (1 / n) * X.T @ (f-y) 

def fit(x,y,w,lr,maxi=1e4,e=0.001):
    w = w.reshape(-1,1)
    wt = w.copy() # previous weights 
    t=0
    while t < maxi:
        g = grad(x,y,wt)
        w = wt - lr * g
        if np.all((np.abs(w - wt) / (np.abs(wt) + 1e-10)) <= e):
            break
        wt = w.copy()
        t += 1
    print(t)
    return w 

In [240]:
# (b) train a model 

x = loan[['age','ed','employ','address','income','debtinc','creddebt','othdebt']].to_numpy()
y = loan[['default']].to_numpy()
w = np.zeros(x.shape[1]+1)
lr = 0.7
w = fit(x,y,w,.01)
print(w)

10000
[[ 0.05542349]
 [ 0.05850076]
 [-0.53071892]
 [-0.1770128 ]
 [-0.01413081]
 [ 0.03944263]
 [ 1.70449611]
 [ 0.31311   ]
 [-0.20761935]]


In [241]:
# (c) 

n = x.shape[0]
X = np.hstack((x,np.ones(n).reshape(-1,1)))
P = sig(X @ w)
P = [1 if i >= 0.5 else 0 for i in p]

i = 0
for j in range(x.shape[0]):
    if P[j] == y[j]:
        i+=1

acc = (i/x.shape[0])*100
print(acc)

26.142857142857146


2. (2 points) Suppose that we have sample data $\mathcal S = \{({\bf x}_i, y_i)\}$, $i=1,\ldots,n$, with each ${\bf x}_i \in \mathbb R^2$. As done in class notes, write the two coordinates of ${\bf x}_i$ as $(x_{i,1}, x_{i,2})$.  Say a logistic model, but with quadratic terms is made &ndash; setting parameters $\omega = (v_1, v_2, v_3, w_1, w_2, b)$, the parameterized function is: $$f_{\omega}({\bf x}_i) = \sigma(v_1x_{i,1}^2 + v_2x_{i,1}x_{i,2} + v_3x_{i,2}^2 + w_1x_{i,1} + w_2x_{i,2} + b).$$
With loss function $\mathcal L_{\mathcal S}$ (as in the slides), write out the partial derivatives $\frac{d}{dv_i}\mathcal L_{\mathcal S}$, $i=1,2,3$.

3. (9 points) We'll show that the per-example loss, when working with a logistic model, is a convex function of $\omega$. Recall that for $\omega = ({\bf w}, b)$ we set $f_{\omega}({\bf x}) = \sigma({\bf w}\cdot{\bf x}+b)$, where $\sigma$ is the logistic function. We'll show that both 
    $$g_1(\omega) = -\log(f_{\omega}({\bf x})) \quad\text{and}\quad  g_0(\omega) = -\log(1-f_{\omega}({\bf x})) $$
    <span margin-left:4em>are convex functions.</span> A function $g:\mathbb R^m\to \mathbb R$ is **convex** if and only if, for all $z_1, z_2\in\mathbb R^m$,
    $$\nabla g(z_1) \cdot (z_2 - z_1) \le g(z_2) - g(z_1).$$
    <span margin-left:4em>In the case that</span> $m=1$, this says that $g'(z_1)\cdot(z_2-z_1) \le g(z_2)-g(z_1)$.

> To show they are convex, fill in the following three steps.

* **(a)** Fixing a point ${\bf x}_i$ from the sample data, first define $h(\omega) = {\bf w}\cdot{\bf x}_i + b$. Compute $\nabla h$ (_note that partials are w.r.t. parameters_), and explain why $$\nabla h(\omega_1) \cdot (\omega_2 - \omega_1) = h(\omega_2) - h(\omega_1)$$ 
for any pair $\omega_1, \omega_2$. And so, $h$ is convex.

* **(b)** Second, let $F:\mathbb R\to\mathbb R$ be (a twice differentiable function) such that $F''(z) > 0$ for all $z\in\mathbb R$. Show that $F$ is convex. 

&emsp;&emsp; (**Hint:** the assumption means that $F'(z)$ is an increasing function. Recall the _Mean Value Theorem_ from Calculus. Can you use it to get that $F'(z_1)\cdot(z_2-z_1) \le F(z_2)-F(z_1)$ for any $z_2>z_1$? What about when $z_2<z_1$?)

* **(c)** Finally, set $F_1(z) = -\log(\sigma(z))$ and $F_0(z) = -\log(1-\sigma(z))$, where $\sigma$ is the logistic function. Check that $F_1''(z) > 0$ and $F_0''(z) > 0$ for all $z$.

Now, $g_1(\omega) = F_1(h(\omega))$ and $g_0(\omega) = F_0(h(\omega))$. A composition of convex functions is a convex function (you don't need to show this), and so these functions are convex.